In [ ]:
from mindnlp.utils import http_get

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

In [ ]:
from mindspore.dataset import TextFileDataset

dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

In [ ]:
train_dataset, test_dataset = dataset.split([0.9, 0.1], randomize=False)

In [ ]:
# article: [CLS] xxxxx [SEP]
# summary: [CLS] xxxxx [SEP]

In [ ]:
import json
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=6, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized[0], tokenized[1], tokenized[0]
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'attention_mask', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [ ]:
from mindnlp.transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4)

In [ ]:
next(train_dataset.create_tuple_iterator())

In [ ]:
len(tokenizer)

In [ ]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel
from mindnlp._legacy.amp import auto_mixed_precision

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2LMHeadModel(config)

optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=1e-4)

In [ ]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

In [ ]:
from mindnlp.engine import Trainer
from mindnlp.engine.callbacks import CheckpointCallback

ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt2_summarization',
                                epochs=1, keep_checkpoint_max=2)

trainer = Trainer(network=model, train_dataset=train_dataset,
                  epochs=5, optimizer=optimizer, callbacks=ckpoint_cb)
trainer.set_amp(level='O1')

In [ ]:
trainer.run(tgt_columns="labels")

In [ ]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized[0]

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [ ]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

In [ ]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=100)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 10:
        break